In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import japanize_matplotlib
import os

In [ ]:
import re

def get_model_select(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    pattern_task_info=r"(\d+\.?\d*) ([A-Za-z_]+) ([a-z_]+)"
    pattern_selected_model=r"Selected model：(.+), val_loss：(\d+\.?\d*)"

    task_info_list=[]
    selected_model_list=[]
    for line in lines:
        match_ti=re.match(pattern_task_info,line)
        if match_ti:
            task_info_list.append((float(match_ti.group(1)),match_ti.group(2),match_ti.group(3)))
        match_ml=re.match(pattern_selected_model,line)
        if match_ml:
            selected_model_list.append((match_ml.group(1),float(match_ml.group(2))))
    
    if len(task_info_list)!=len(selected_model_list):
        print("ERROR")
        return None

    result_model_selection={}
    for i in range(len(task_info_list)):
        if not (task_info_list[i][1] in result_model_selection.keys()):
            result_model_selection[task_info_list[i][1]]=[]

        # (Execution mode, selected model, model's val_loss)
        result_model_selection[task_info_list[i][1]].append((task_info_list[i][2],selected_model_list[i][0],selected_model_list[i][1]))
    return result_model_selection

In [ ]:
def get_max_array(result_path,result_num=10):
    result_list=[]
    for i in range(0,result_num):
        df = pd.read_csv(os.path.join(result_path,f"observed_target{i}.csv"), header=None)
        result_list.append(df)
    searched_array = pd.concat(result_list, axis=1).to_numpy()
    
    max_array = np.zeros(searched_array.shape)

    for i in range(max_array.shape[1]):
        for j in range(max_array.shape[0]):
            max_array[j,i]=searched_array[0:j+1,i].max()
    
    return max_array, searched_array

In [ ]:
def plot_mean_max_aray(result_path, label, color, obj_name, result_num=10, random_search=10):
    max_array,_ = get_max_array(result_path,result_num)
    mean_max_array = max_array.mean(axis=1)

    plt.plot(max_array, c=color, alpha=0.1)
    plt.plot(mean_max_array, c=color, label=label)

    # Points where knowledge sharing was performed
    # result_list=get_model_select(os.path.join(os.path.dirname(result_path),"stdout0"))[f"BO_system_{obj_name}"]
    # for j in range(len(result_list)):
    #     if result_list[j][0]=="transfer":
    #         plt.axvline(x=random_search+j, color='gray', linestyle='--', alpha=0.2)
    # Display the start of exploration
    plt.axvline(x=random_search, color='gray', linestyle='-', alpha=0.5)

    obj_dict={"CT":6,"MM":0,"SP":2}
    full_data = pd.read_csv("/workspace/BO_system/dataset/B2_20220520_full.csv",header=None)
    OBJ_MAX = full_data[obj_dict[obj_name]].max()
    plt.plot(np.array([OBJ_MAX for i in range(max_array.shape[0])]),"--",c="g")

    print(label,":",mean_max_array.mean())

In [ ]:
def plot_compare(result_path1, result_path2, label_list, obj_name, result_num=10):
    max_array1,searched_array1=get_max_array(result_path1, result_num)
    max_array2,searched_array2=get_max_array(result_path2, result_num)

    obj_dict={"CT":6,"MM":0,"SP":2}
    full_data = pd.read_csv("/workspace/BO_system/dataset/B2_20220520_full.csv",header=None)
    OBJ_MAX = full_data[obj_dict[obj_name]].max()

    for i in range(result_num):
        plt.plot(np.array([OBJ_MAX for i in range(max_array1.shape[0])]),"--",c="g")
        plt.plot(max_array1[:,i], c="C0", label=label_list[0])
        plt.plot(max_array2[:,i], c="C1", label=label_list[1])
        # Plot exploration points
        plt.plot(searched_array1[:,i], c="C0", marker=".", linestyle="None")
        plt.plot(searched_array2[:,i], c="C1", marker=".", linestyle="None")

        # Moving average of the past n iterations
        # ma_n=10
        # moving_average1=np.array([searched_array1[(j-ma_n+1 if j-ma_n+1>=0 else 0):j+1,i].mean() for j in range(searched_array1.shape[0])])
        # moving_average2=np.array([searched_array2[(j-ma_n+1 if j-ma_n+1>=0 else 0):j+1,i].mean() for j in range(searched_array2.shape[0])])
        # plt.plot(moving_average1, c="C0", linestyle="-")
        # plt.plot(moving_average2, c="C1", linestyle="-")

        # Points where knowledge sharing was performed
        for j in range(15, max_array1.shape[0], 6):
            plt.axvline(x=j, color='gray', linestyle='--', alpha=0.2)
        # Display the start of exploration
        plt.axvline(x=10, color='gray', linestyle='-', alpha=0.5)
        plt.title(f"Comparison of Optimization Efficiency {i} ({obj_name})")
        plt.xlabel("Number of Explorations")
        plt.ylabel(f"Maximum {obj_name} Found")
        plt.legend(loc="lower right")
        plt.show()

### Comparison of average optimization results

In [ ]:
plot_mean_max_aray("result_network10_6/BO_system_CT","No Knowledge Sharing","C0", "CT", result_num=10, random_search=10)
plot_mean_max_aray("result_network9_6/BO_system_CT","With Knowledge Sharing","C1", "CT", result_num=10, random_search=10)

plt.title("Comparison of CT Optimization Efficiency")
plt.xlabel("Number of Explorations")
plt.ylabel("Maximum CT Found")
plt.legend(loc="lower right")
# plt.xlim(1000-10,1100)
# plt.ylim(800,1000)
plt.show()

In [ ]:
plot_mean_max_aray("result_network10_6/BO_system_MM","No Knowledge Sharing","C0", "MM", result_num=10, random_search=10)
plot_mean_max_aray("result_network9_6/BO_system_MM","With Knowledge Sharing","C1", "MM", result_num=10, random_search=10)

plt.title("Comparison of MM Optimization Efficiency")
plt.xlabel("Number of Explorations")
plt.ylabel("Maximum MM Found")
plt.legend(loc="lower right")
# plt.xlim(2000-10,2200)
# plt.xlim(2000-10,2200)
# plt.ylim(2.4,2.7)
plt.show()

In [ ]:
plot_mean_max_aray("result_network10_6/BO_system_SP","No Knowledge Sharing","C0", "SP", result_num=10, random_search=10)
plot_mean_max_aray("result_network9_6/BO_system_SP","With Knowledge Sharing","C1", "SP", result_num=10, random_search=10)

plt.title("Comparison of SP Optimization Efficiency")
plt.xlabel("Number of Explorations")
plt.ylabel("Maximum SP Found")
plt.legend(loc="lower right")
plt.show()